In [1]:
import numpy as np
import pandas as pd

from skimage import io
from skimage.filters import threshold_otsu

import os
import napari
import pickle
import xarray as xr

import morphotrack.points
import morphotrack.track
import morphotrack.image

/home/tmurakami/app/miniconda3/envs/segment/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [3]:
io_directory = '/mnt/ampa_data01/tmurakami/220715_prefrontal_q2_R01/morphotrack'
# load blood vessel
vessel = io.imread(os.path.join(io_directory,'R01_ch561_10um.tif'))

# load nuclei
nuclei = io.imread(os.path.join(io_directory,'R02_ch488_10um.tif'))

# load a mask of a white matter 
white_matter = io.imread(os.path.join(io_directory,'white_matter.tif'))!=0

# load a mask of a layer 1
layer1 = io.imread(os.path.join(io_directory,'l1.tif'))!=0

# segmentated tissue
segmented_tissue = io.imread(os.path.join(io_directory,'tissue_mask.tif'))!=0

# read cellular coordinate
cells_info = pd.read_csv('/mnt/ampa_data01/tmurakami/220715_prefrontal_q2_R01/R02_R01/R02ch488_to_R01_segmentation/coord_centroids.csv',index_col=0) # the unit is original voxel.
scaling_factor = np.asarray([3,0.65,0.65])
cell_coords = cells_info.loc[:,['centroid-0','centroid-1','centroid-2']] * scaling_factor / 10# convert unit to 10 um for consistency
cell_coords = cell_coords.to_numpy()

# load regression model
reg = pickle.load(open(os.path.join(io_directory,'model_debug2.pkl'), 'rb'))
norm_flow_on_coordinate = morphotrack.points.model_to_norm_flow_func(reg.degree, reg)

In [4]:
# visual sanity check
viewer= napari.Viewer()
viewer.add_image(layer1, blending='additive')
viewer.add_image(white_matter, blending='additive')
viewer.add_image(nuclei, blending='additive')
viewer.add_image(segmented_tissue, blending='additive')

<Image layer 'segmented_tissue' at 0x7f62583bbfd0>

In [4]:
# selection of the outer most cells.
# convex hull is not successful. Try alpha shape instead.
# the algorithm does not run in parallel
ver, nor = morphotrack.points.cloud_to_alphashape(cell_coords, downsample=10, alpha=0.1, return_normal=True)

CPU times: user 4min 33s, sys: 1.19 s, total: 4min 34s
Wall time: 4min 35s


In [62]:
# selection of the points using clustering
selection = morphotrack.points.face_selection(ver, nor, norm_flow_on_coordinate(ver), n_clusters=6, cluster_selection='min')
points = ver[selection]

viewer = napari.Viewer()
viewer.add_image(nuclei, rgb=False, name='nuclei', colormap='gray', blending='additive')
viewer.add_points(
    ver,
    size=5,
    face_color = 'green',
    edge_width=0,
    symbol='disc',
)

viewer.add_points(
    points,
    size=5,
    face_color = 'white',
    edge_width=0,
    symbol='disc',
)

/home/tmurakami/app/miniconda3/envs/segment/lib/python3.8/site-packages/numpy/core/numeric.py:2449: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

/home/tmurakami/app/miniconda3/envs/segment/lib/python3.8/site-packages/numpy/core/numeric.py:2449: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



<Points layer 'points' at 0x7f7cbc260bb0>

## Seeding and tracking

In [24]:
polynomial_vectorfield = morphotrack.track.polynomial_vectorfield_generator(reg)
tracker = morphotrack.track.FieldTracker(polynomial_vectorfield, points) # make a FieldTracker object

# tracking
t_start = 0
t_end = 500
tracker.solve_ode(t_start,t_end)

In [25]:
# Visualize tracks
position = tracker.t_positions.stack(pos=['time', 'track']).data.T
position[np.isnan(position)] = 0
flow_vector = abs(norm_flow_on_coordinate(position))

flow_img = np.zeros(vessel.shape+(vessel.ndim,))
for i in range(vessel.ndim):
    flow_img[...,i] = morphotrack.image.fill_value_in_range(flow_img[...,i],position.T,flow_vector[:,i])
    
viewer = napari.Viewer()
viewer.add_image(vessel, contrast_limits=[0,20000], rgb=False, name='sma', colormap='gray', blending='additive')

palette = [
    Colormap([[1.0, 1.0, 1.0, 0.0], [1.0, 0.0, 0.0, 1.0]]),
    Colormap([[1.0, 1.0, 1.0, 0.0], [0.0, 1.0, 0.0, 1.0]]),
    Colormap([[1.0, 1.0, 1.0, 0.0], [0.0, 0.0, 1.0, 1.0]])]

for i in range(vessel.ndim):
    viewer.add_image(flow_img[...,i], contrast_limits=[0,1], rgb=False, name=i, colormap=palette[i], blending='translucent')

<Image layer 'mask' at 0x7f7cb5779ee0>

In [57]:
tracks = tracker.t_positions
flow = tracker.apply_function_to_position(norm_flow_on_coordinate)
seeds = tracker.seeds_xr

ds = xr.Dataset(data_vars={'Position': tracks, 'Flow': flow, 'Seed': seeds})

In [61]:
ds.to_netcdf(os.path.join(io_directory,"track_info.nc"))